# Azure functions の例

> 注意: openaiライブラリの新しいバージョンが利用可能です。https://github.com/openai/openai-python/discussions/742 を参照してください

このノートブックでは、Azure OpenAIサービスでfunction calling機能を使用する方法を示します。Functionsを使用することで、チャット補完の呼び出し元は、モデルが外部ツールやデータソースに機能を拡張するために使用できる機能を定義できます。

チャットfunctionsについて詳しくは、OpenAIのブログをお読みください：https://openai.com/blog/function-calling-and-other-api-updates

**注意**: チャットfunctionsには、gpt-4およびgpt-35-turboの`-0613`ラベルから始まるモデルバージョンが必要です。古いバージョンのモデルではサポートされていません。

## セットアップ

まず、必要な依存関係をインストールします。

In [ ]:
! pip install "openai>=0.28.1,<1.0.0"
# (Optional) If you want to use Microsoft Active Directory
! pip install azure-identity

In [3]:
import os
import openai

さらに、Azure OpenAI Serviceに適切にアクセスするために、[Azure Portal](https://portal.azure.com)で適切なリソースを作成する必要があります（これを行う方法の詳細なガイドは[Microsoft Docs](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)で確認できます）。

リソースが作成されたら、最初に使用する必要があるのはそのエンドポイントです。エンドポイントは、*「リソース管理」*セクション内の*「キーとエンドポイント」*セクションを確認することで取得できます。これを取得したら、この情報を使用してSDKをセットアップします：

In [27]:
openai.api_base = "" # Add your endpoint here

# functions is only supported by the 2023-07-01-preview API version
openai.api_version = "2023-07-01-preview"

### 認証

Azure OpenAIサービスは、APIキーとAzure認証情報を含む複数の認証メカニズムをサポートしています。

In [4]:
use_azure_active_directory = False

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_type`を`azure`に設定し、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の下の*「キーとエンドポイント」*で確認できます）。

In [ ]:
if not use_azure_active_directory:
    openai.api_type = "azure"
    openai.api_key = os.environ["OPENAI_API_KEY"]

> 注意: この例では、コード内で変数を設定することでライブラリがAzure APIを使用するように設定しました。開発時には、代わりに環境変数を設定することを検討してください：

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

#### Microsoft Active Directoryを使用した認証
次に、Microsoft Active Directory認証を使用してキーを取得する方法を見てみましょう。

In [ ]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = "azure_ad"
    openai.api_key = token.token

トークンには有効期限があり、期限が過ぎると無効になります。すべてのリクエストで有効なトークンが送信されるようにするため、`requests.auth`にフックして期限切れのトークンを更新することができます：

In [ ]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

if use_azure_active_directory:
    session = requests.Session()
    session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session

## 関数

セットアップと認証が完了したので、Azure OpenAIサービスで関数を使用できるようになりました。これはいくつかのステップに分かれています：

1. 関数を定義する
2. 関数定義をchat completions APIに渡す
3. レスポンスからの引数を使って関数を呼び出す
4. 関数のレスポンスをchat completions APIにフィードバックする

#### 1. 関数の定義

関数のリストを定義できます。各関数には、関数名、オプションの説明、および関数が受け取るパラメータ（JSONスキーマとして記述）が含まれます。

In [21]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location"],
        },
    }
]

#### 2. 関数定義をchat completions APIに渡す

これで関数をchat completions APIに渡すことができます。モデルが関数を呼び出すべきと判断した場合、choiceに`finish_reason`として"function_call"が設定され、どの関数を呼び出すかとその引数の詳細が`message`に含まれます。オプションとして、`function_call`キーワード引数を設定することで、モデルに特定の関数を強制的に呼び出させることができます（例：`function_call={"name": get_current_weather}`）。デフォルトでは、これは`auto`に設定されており、モデルが関数を呼び出すかどうかを選択できるようになっています。

In [ ]:
messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "What's the weather like today in Seattle?"}
]

chat_completion = openai.ChatCompletion.create(
    deployment_id="gpt-35-turbo-0613",
    messages=messages,
    functions=functions,
)
print(chat_completion)

{
  "choices": [
    {
      "content_filter_results": {},
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "function_call": {
          "arguments": "{\n  \"location\": \"Seattle, WA\"\n}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1689702512,
  "id": "chatcmpl-7dj6GkYdM7Vw9eGn02bc2qqjN70Ps",
  "model": "gpt-4",
  "object": "chat.completion",
  "prompt_annotations": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "prompt_index": 0
    }
  ],
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 115,
    "total_to

#### 3. レスポンスから引数を使って関数を呼び出す

関数呼び出しの名前は最初に提供されたもののいずれかになり、引数には関数定義に含まれるスキーマに一致するJSONが含まれます。

In [ ]:
import json

def get_current_weather(request):
    """
    This function is for illustrative purposes.
    The location and unit should be used to determine weather
    instead of returning a hardcoded response.
    """
    location = request.get("location")
    unit = request.get("unit")
    return {"temperature": "22", "unit": "celsius", "description": "Sunny"}

function_call =  chat_completion.choices[0].message.function_call
print(function_call.name)
print(function_call.arguments)

if function_call.name == "get_current_weather":
    response = get_current_weather(json.loads(function_call.arguments))

get_current_weather
{
  "location": "Seattle, WA"
}


#### 4. 関数のレスポンスをchat completions APIにフィードバックする

関数からのレスポンスは、ロールを"function"に設定した新しいメッセージにシリアライズする必要があります。これで、モデルはレスポンスデータを使用して回答を作成します。

In [ ]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": json.dumps(response)
    }
)

function_completion = openai.ChatCompletion.create(
    deployment_id="gpt-35-turbo-0613",
    messages=messages,
    functions=functions,
)

print(function_completion.choices[0].message.content.strip())

Today in Seattle, the weather is sunny with a temperature of 22 degrees celsius.
